In [ ]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import numpy as np
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import shap


drive.mount('/content/gdrive')
folder ='gdrive/MyDrive/IMU_LOSO'
contents = os.listdir(folder)

##for el in contents:
 # if "txt" in el:
 #   file_path = os.path.join(folder, el)

 #   if os.path.exists(file_path):
 #     f = open(file_path, "r")
 #     f.readline()
 #     for row in f.readlines():
 #       print(row)

def load_data():
    Lista_Dati = []
    Lista_label = []
    Lista_Soggetto=[]
    file_list = os.listdir(".")
    for filename in contents:
        if "txt" in filename:
            file_path = os.path.join(folder, filename)
            f = open(file_path, "r")
            for row in f.readlines():
                Row_S = row.strip().split(",")
                #print(Row_S)
                # print(len(Row_S))
                Lista_Dati.append(
                    (float(Row_S[1]), float(Row_S[2]), float(Row_S[3]), float(Row_S[4]), float(Row_S[5]),
                     float(Row_S[6]),
                     float(Row_S[7]), float(Row_S[8]), float(Row_S[9]), float(Row_S[10]),
                     float(Row_S[11]), float(Row_S[12]), float(Row_S[13]), float(Row_S[14]), float(Row_S[15]),
                     float(Row_S[16]),
                     float(Row_S[17]), float(Row_S[18]), float(Row_S[19]),
                     float(Row_S[20]), float(Row_S[21]), float(Row_S[22]), float(Row_S[23]), float(Row_S[24])))
                Lista_label.append(Row_S[25])
                Lista_Soggetto.append(Row_S[26])

    return Lista_Dati, Lista_label, Lista_Soggetto

#load_data()

Lista_dati, Lista_label, Lista_Soggetto = load_data()
print(len(Lista_dati))


def Divido_per_soggetto(Lista_dati, Lista_label, Lista_Soggetto):
    Training =[]
    Test = []
    Label_Training = []
    Label_Test = []
    for i in range(len(Lista_Soggetto)):
        if Lista_Soggetto[i] == "AB27" or Lista_Soggetto[i] == "AB28" or Lista_Soggetto[i] == "AB30":
            Test.append(Lista_dati[i])
            Label_Test.append(Lista_label[i])
        else:
            Training.append(Lista_dati[i])
            Label_Training.append(Lista_label[i])

    return Training, Test, Label_Training, Label_Test


X,Y,x_label,y_label =Divido_per_soggetto(Lista_dati,Lista_label,Lista_Soggetto)


def finestre_temporali(Lista_dati, Lista_label):
    Grouped_List = []
    Lista_label_ridotta = []
    for i in range(0, len(Lista_dati), 100):
        group = Lista_dati[i:i + 100]
        group2 = Lista_label[i:i + 100]
        Grouped_List.append(group)
        Lista_label_ridotta.append(group2)
   # print(Grouped_List)
    #print(Lista_label_ridotta)
    Lista_indici = []
    Lista_dati_Corretta = []
    for i in range(len(Lista_label_ridotta)):
        #print(Lista_label_ridotta[i])
        if len(set(Lista_label_ridotta[i])) < 2:
            Lista_indici.append(set(Lista_label_ridotta[i]))
            Lista_dati_Corretta.append(Grouped_List[i])

    Lista_Indici_Finale = []
    for el in Lista_indici:
        lista = list(el)
        for el2 in lista:
            Lista_Indici_Finale.append(el2)

    return Lista_dati_Corretta, Lista_Indici_Finale


X2, X2_label =finestre_temporali(X, x_label)

Y2, Y2_label = finestre_temporali(Y, y_label)


## Senza Label ##
def do_label_numerica(X_1,y_1):
    X_new = []
    y_new = []
    for i in range(len(y_1)):
        if y_1[i] == "idle":
            continue
        else:
            X_new.append(X_1[i])
            y_new.append(y_1[i])

    return X_new, y_new


X2_new, X2_label_new = do_label_numerica(X2, X2_label)

Y2_new, Y2_label_new = do_label_numerica(Y2, Y2_label)


def do_label_Corretta(Nuove):
    Y_new2 = []
    for el in Nuove:
        if el =="walk-stairascent" or el =="stairascent" or el =="stairascent-walk":
            Y_new2.append(0)
        elif el == "walk-stairdescent" or el =="stairdescent" or el =="stairdescent-walk":
            Y_new2.append(1)
        elif el == "walk-rampascent" or el =="rampascent" or el =="rampascent-walk":
            Y_new2.append(2)
        elif el == "walk-rampdescent" or el =="rampdescent" or el =="rampdescent-walk":
            Y_new2.append(3)
        else:
            Y_new2.append(4)

    return Y_new2

Train_label = do_label_Corretta(X2_label_new)
Test_label = do_label_Corretta(Y2_label_new)

def creo_numpy_array(Y_new2C, X_new):
    New_X = []
    New_y = []
    for i in range(len(X_new)):
        if len(X_new[i]) < 100:
            continue
        else:
            New_X.append(X_new[i])
            New_y.append(Y_new2C[i])

    New_X = np.array(New_X)
    New_y = np.array(New_y)
    return New_X, New_y

Dati_X, label_x = creo_numpy_array(Train_label, X2_new)
Dati_test_y, label_test_y = creo_numpy_array(Test_label, Y2_new)

model = keras.Sequential([
    keras.layers.LSTM(128, return_sequences=True, input_shape=(100, 24)),  # LSTM layer with return sequences
    keras.layers.LSTM(128, return_sequences=False),  # Another LSTM layer
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(5, activation='softmax')  # 3 output classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(Dati_X, label_x, epochs=10)


#model.save('Prova.h5')

loss, accuracy = model.evaluate(Dati_test_y, label_test_y)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')
predictions = model.predict(Dati_test_y)

predicted_labels = predictions.argmax(axis=1)
accuracy = accuracy_score(label_test_y, predicted_labels)
precision = precision_score(label_test_y, predicted_labels, average='weighted')
recall = recall_score(label_test_y, predicted_labels, average='weighted')
f1 = f1_score(label_test_y, predicted_labels, average='weighted')
conf_matrix = confusion_matrix(label_test_y, predicted_labels)

# Printing the metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')


#feature_names = ["foot_Accel_X", "foot_Accel_Y","foot_Accel_Z",
#           "foot_Gyro_X", "foot_Gyro_Y", "foot_Gyro_Z",
#           "shank_Accel_X","shank_Accel_Y","shank_Accel_Z",
#           "shank_Gyro_X","shank_Gyro_Y","shank_Gyro_Z",
#           "thigh_Accel_X","thigh_Accel_Y","thigh_Accel_Z",
#           "thigh_Gyro_X","thigh_Gyro_Y","thigh_Gyro_Z",
#           "trunk_Accel_X","trunk_Accel_Y","trunk_Accel_Z",
#           "trunk_Gyro_X","trunk_Gyro_Y","trunk_Gyro_Z"]

feature_names = ["f_A_X", "f_A_Y","f_A_Z",
           "f_G_X", "f_G_Y", "f_G_Z",
           "s_A_X","s_A_Y","s_A_Z",
           "s_G_X","s_G_Y","s_G_Z",
           "th_A_X","th_A_Y","th_A_Z",
           "th_G_X","th_G_Y","th_G_Z",
           "tr_A_X","tr_A_Y","tr_A_Z",
           "tr_G_X","tr_G_Y","tr_G_Z"]

# Use GradientExplainer with the model itself
explainer = shap.GradientExplainer(model, Dati_X)
shap_values = explainer.shap_values(Dati_test_y)
print(shap_values)

cls = 0
idx = 0
shap.summary_plot(shap_values[cls][:,idx,:], Dati_test_y[:,idx,:],plot_type="bar", feature_names =feature_names)


custom_feature_order = ["f_A_X", "f_A_Y","f_A_Z",
           "f_G_X", "f_G_Y", "f_G_Z",
           "s_A_X","s_A_Y","s_A_Z",
           "s_G_X","s_G_Y","s_G_Z",
           "th_A_X","th_A_Y","th_A_Z",
           "th_G_X","th_G_Y","th_G_Z",
           "tr_A_X","tr_A_Y","tr_A_Z",
           "tr_G_X","tr_G_Y","tr_G_Z"]

feature_indices = [feature_names.index(feature) for feature in custom_feature_order]
# Reorder SHAP values and feature values
shap_values_reordered = shap_values[cls][:, idx, feature_indices]
print("SonoQua")
print(shap_values_reordered)
print(feature_indices)
#X_reordered = X[:, idx, feature_indices]
X_reordered = Dati_test_y[:, idx, feature_indices]

# Calculate the absolute mean of SHAP values
abs_mean_shap_values = np.mean(np.abs(shap_values[cls][:, idx, :]), axis=0)
num_features = len(custom_feature_order)

# Create a list of colors for each set of six bars
bar_colors = ['green', 'green', 'green', 'green', 'green','green',
              'red', 'red', 'red', 'red', 'red','red',
              'blue','blue','blue','blue','blue','blue',
              'orange','orange','orange','orange','orange','orange'] * (num_features // 24)
# Create a bar chart
plt.figure(figsize=(20, 8))
plt.bar(custom_feature_order, abs_mean_shap_values,color=bar_colors)
plt.xlabel('Feature')
plt.ylabel('Absolute Mean SHAP Value')
plt.title('Absolute Mean SHAP Values')
plt.xticks(rotation='vertical')
mean_line_1 = np.mean(abs_mean_shap_values[:6])
mean_line_2 = np.mean(abs_mean_shap_values[6:12])
mean_line_3 = np.mean(abs_mean_shap_values[12:18])
mean_line_4 = np.mean(abs_mean_shap_values[18:24])
plt.axhline(mean_line_1, color='green', linestyle='--', label='Mean Foot')
plt.axhline(mean_line_2, color='red', linestyle='--', label='Mean Shank')
plt.axhline(mean_line_3, color='blue', linestyle='--', label='Mean Thigh')
plt.axhline(mean_line_4, color='orange', linestyle='--', label='Mean Trunk')


plt.legend()
plt.show()
